# FPL Team Selector: Git Branch and Agent Attribution Demo

*Created by GitHub Copilot agent*


## 1. Set Up Git Repository
This section ensures you are in the correct Git repository. If you haven't initialized a repository, run the following command in your project directory:

```sh
git init
```
If you already have a repository, you can skip this step.

In [ ]:
# Navigate to your project directory (if not already there)
import os
os.chdir(r'C:/Git/IDWT-All-Things-SQL-And-Python-And-Code')

# Initialize git repository if needed
!git status || git init

## 2. Create a New Branch
Create and switch to a new branch for your changes. Replace `<branch_name>` with your desired branch name. Example: `copilot-fpl-agent-update`.

```sh
git checkout -b copilot-fpl-agent-update
```

In [ ]:
# Create and switch to a new branch
branch_name = 'copilot-fpl-agent-update'
!git checkout -b {branch_name}

## 3. Modify a File
Add the string 'created my agent from copilot' to a name in a file. For demonstration, we'll update a file called `agent_name.txt`.

In [ ]:
# Modify a file by adding the agent attribution
filename = 'agent_name.txt'
name = 'FPL Team Selector'
with open(filename, 'w') as f:
    f.write(f'{name} - created my agent from copilot\n')
print(f"Updated {filename} with attribution.")

## 4. Commit Changes
Stage and commit your changes to the new branch. Example commit message: 'Add agent attribution to name'.

```sh
git add agent_name.txt
git commit -m "Add agent attribution to name"
```

In [ ]:
# Stage and commit the changes
!git add agent_name.txt
!git commit -m "Add agent attribution to name"

## 5. Push the Branch to Remote
Push your new branch to the remote repository. Make sure you have the correct remote set up (usually 'origin').

```sh
git push -u origin copilot-fpl-agent-update
```

In [ ]:
# Push the branch to remote
!git push -u origin copilot-fpl-agent-update

## 6. FPL Team Selection: Fetch Data and Optimize Team
This section fetches the latest FPL data from the public API, runs a linear programming optimization to select the optimal team, and displays a sample output.

In [1]:
# Install required packages (if not already installed)
import sys
!{sys.executable} -m pip install requests pulp pandas

^C


In [ ]:
import requests
import pandas as pd
import pulp as lp

# Fetch latest FPL data
fpl_api_url = "https://fantasy.premierleague.com/api/bootstrap-static/"
response = requests.get(fpl_api_url)
data = response.json()

# Prepare player and team data
elements = data["elements"]
teams = data["teams"]

players_data = {}
teams_data = {}
for player in elements:
    player_name = f"{player['first_name']} {player['second_name']}"
    players_data[player_name] = {
        "position": player["element_type"],
        "points_total": player["total_points"],
        "cost": player["now_cost"] / 10,  # Convert to millions
        "team": player["team"],
    }
    team_id = player["team"]
    team_name = teams[team_id - 1]["name"]  # Adjust team_id to index
    teams_data[team_id] = team_name

# FPL constraints
max_starting_11_players_per_position = {1: 1, 2: 5, 3: 5, 4: 3}
min_starting_11_players_per_position = {1: 1, 2: 3, 3: 2, 4: 1}
max_squad_players_per_position = {1: 2, 2: 5, 3: 5, 4: 3}
total_budget = 100

# Optimization model
model = lp.LpProblem("FantasyFootballTeam", lp.LpMaximize)
player_vars_starting_11 = {player: lp.LpVariable(f"starting_11_{player}", cat=lp.LpBinary) for player in players_data}
player_vars_squad = {player: lp.LpVariable(f"squad_{player}", cat=lp.LpBinary) for player in players_data}

# Objective: maximize total points from starting 11
model += lp.lpSum(players_data[player]["points_total"] * player_vars_starting_11[player] for player in players_data), "TotalPointsStarting11"

# Constraints
model += lp.lpSum(players_data[player]["cost"] * player_vars_squad[player] for player in players_data) <= total_budget, "Budget"
model += lp.lpSum(player_vars_squad[player] for player in players_data) == 15, "SquadSize"
for player in players_data:
    model += player_vars_starting_11[player] <= player_vars_squad[player], f"Starting11InSquad_{player}"
model += lp.lpSum(player_vars_starting_11[player] for player in players_data) == 11, "Starting11Size"
for position, min_count in min_starting_11_players_per_position.items():
    model += lp.lpSum(player_vars_starting_11[player] for player in players_data if players_data[player]["position"] == position) >= min_count, f"MinStarting11PlayersPosition{position}"
for position, max_count in max_starting_11_players_per_position.items():
    model += lp.lpSum(player_vars_starting_11[player] for player in players_data if players_data[player]["position"] == position) <= max_count, f"MaxStarting11PlayersPosition{position}"
for position, max_count in max_squad_players_per_position.items():
    model += lp.lpSum(player_vars_squad[player] for player in players_data if players_data[player]["position"] == position) <= max_count, f"MaxSquadPlayersPosition{position}"
for team_id, team_name in teams_data.items():
    model += lp.lpSum(player_vars_squad[player] for player in players_data if players_data[player]["team"] == team_id) <= 3, f"MaxPlayersPerTeam_{team_name}"

# Solve
model.solve()

# Extract results
selected_players = [player for player in players_data if lp.value(player_vars_squad[player]) > 0]
starting_11 = [player for player in players_data if lp.value(player_vars_starting_11[player]) > 0]
bench_players = [player for player in selected_players if player not in starting_11]

# Suggest captain (highest points in starting 11)
captain = max(starting_11, key=lambda x: players_data[x]["points_total"])

# Output
print("Sample Output: Optimal FPL Team")
print("\nStarting 11:")
for player in starting_11:
    data = players_data[player]
    print(f"{player} | Position: {data['position']} | Points: {data['points_total']} | Cost: £{data['cost']}m | Team: {teams_data[data['team']]}")
print("\nBench:")
for player in bench_players:
    data = players_data[player]
    print(f"{player} | Position: {data['position']} | Points: {data['points_total']} | Cost: £{data['cost']}m | Team: {teams_data[data['team']]}")
print(f"\nSuggested Captain: {captain}")